Libraries Used:

Librosa: http://librosa.github.io/


In [46]:
import librosa
from ly import *
import os


In [47]:
def xml_to_ly(filepath, p = False):
    '''
    INPUTS: 
    filepath
    '''
    file = './dataset/'+filepath
    cmd = 'musicxml2ly -a '+file
    returned_value = os.system(cmd)
    if returned_value != 0:
        raise ValueError('File Not Found')
    if p ==  True:
        print('returned value:', returned_value)




In [60]:
def ly_to_text(filepath):
    '''
    INPUTS:    
        filepath: string to file path of .ly file
    -----------------------------------------
    RETURNS:
        data: string of encoded text from .ly file
    '''
    with open(ly_file,encoding='cp1252') as f: #Might have to change encoding depending on how the mac version is
        data = f.read()
    return data

All nodes (instances of Item) have a ‘position’ attribute that indicates where the item starts in the source text. Almost
all items have the token that starts the expression in the ‘token’ attribute and possibly other tokens in the ‘tokens’
attribute, as a tuple.
The ‘end_position()’ method returns the position where the node (including its child nodes) ends.

In [63]:
filename = "Conga_Double_Stroke_Exercise"
xml_file = filename+ ".mxl"
ly_file = filename+".ly"

data = ly_to_text(ly_file)

print(data)
print(len('PartPOneVoiceOne'))


\version "2.18.2"
% automatically converted by musicxml2ly from ./dataset/Conga_Double_Stroke_Exercise.mxl

\header {
    encodingsoftware = "MuseScore 3.0.2"
    encodingdate = "2019-02-27"
    title = Conga
    }

#(set-global-staff-size 20.0750126457)
\paper {
    paper-width = 21.01\cm
    paper-height = 29.69\cm
    top-margin = 1.0\cm
    bottom-margin = 2.0\cm
    left-margin = 1.0\cm
    right-margin = 1.0\cm
    }
\layout {
    \context { \Score
        autoBeaming = ##f
        }
    }
PartPOneVoiceOne =  {
    \clef "percussion" \key c \major \repeat volta 2 {
        | % 1
        a'4 ^"R" a'4 ^"R" a'4 ^"L" a'4 ^"L" s1 }
    \break \repeat volta 2 {
        | % 3
        a'8 [ a'8 a'8 a'8 ] a'8 [ a'8 a'8 a'8 ] s1 }
    \break \repeat volta 2 {
        | % 5
        \once \override TupletBracket #'stencil = ##f
        \times 2/3  {
            a'8 [ a'8 a'8 ] }
        \once \override TupletBracket #'stencil = ##f
        \times 2/3  {
            a'8 [ a'8 a'8 ] }
       

In [65]:
def parse_PPOVO(bpm, PPOVO):
    '''
    INPUTS:
        bpm: beats per minute integer
        PPOVO: string of ly file containing note information
    --------------------------------
    RETURNS:
        onset_times: array of all float ground truth onset times
    '''
    time = 0.0
    onset_times = []
    prev = None
    i = 0
    while i < len(PPOVO)-1:
        c = PPOVO[i]
        if(c == "'"): #note found
            #increment start by
            i += 1
            c = PPOVO[i]
            buffer = ""
            #keep going until total note len is found (1,2,4,8th,16th,32th note etc)
            while (c.isdisit())
                buffer+=c
                i+=1
                c = PPOVO[i]
                             
            note_len = 1.0/int(buffer)
                
        
        
        
        prev = c
        start +=1
        
    

In [ ]:
def note_to_seconds(bpm, note_val, cur_time):
    '''
    INPUTS: 
        beats per minute integer
        note_val: type of note in float 
            1.0 = whole note
            0.5 = half note
            etc
        cur_time:
            current time of last onset in float
        
    -------------------------------
    RETURNS:
        duration: when in time note onset occured
    '''
    return cur_time + ((60.0/bpm)*

In [ ]:
def ly_onsets(bpm, data):
    '''
    INPUTS:
        bpm: beats per minute integer
        data: string of ly file
    --------------------------------
    RETURNS:
        onset_times: array of floats for time stamps ground truth onsets
    '''
    #skip header information
    start = data.find('PartPOneVoiceOne') 
    #improper file type
    if start == -1:
        raise ValueError('Improper File Format. File is not Monophonic')
        
    #adjust start to begin at PPOVO node
    start +=  16
    while data[start] != '{':
        start += 1
        
    PPOVO = data[start:(len(data)-1)]
    
    #parse through to find note section
    onset_times = parse_PPOVO(bpm, PPOVO)
    
    return onset_times
        
        
    